In [1]:
from tensorflow.keras import layers, losses, models as mds, optimizers
from collections import namedtuple
from random import sample, shuffle
from math import floor

import tensorflow as tf
import tensorflow_datasets as tfds

2022-02-17 21:38:03.686713: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-17 21:38:03.686790: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Dataset image size
IMG_SIZE = 264
N_CLASSES = 102

def resize_image(image, *args):
    image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
    return image, *args

def normalize_image(image, *args):
    image /= 255
    return image, *args

train_ds, val_ds = tfds.load(
    'oxford_flowers102',
    split=['train[:20%]', 'validation'],
    as_supervised=True,
    read_config=tfds.ReadConfig(try_autocache=False)
)

train_ds = train_ds.map(resize_image).map(normalize_image)
val_ds = val_ds.map(resize_image).map(normalize_image)

2022-02-17 21:38:06.935407: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-17 21:38:06.935471: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-17 21:38:06.935499: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-19319V3): /proc/driver/nvidia/version does not exist
2022-02-17 21:38:06.935922: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
itr = iter(train_ds)
next(itr)[0][100, 100]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.34849134, 0.3406482 , 0.28966781], dtype=float32)>

In [4]:
MEMBER_RUNNING_ID = 0

# Member = namedtuple('Member', ['id', 'gen', 'model'])
class Member(object):
    def __init__(self, member_id, parent, gen, model):
        self.id = member_id
        self.parent = parent
        self.gen = gen
        self.model = model
    
    def __repr__(self):
        return '[id: {}, parent: {}, gen: {}, model: {}]'.format(self.id, self.parent, self.gen, self.model)

def init_model():
    model = tf.keras.Sequential([
        tf.keras.Input((IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        layers.GlobalMaxPooling2D(),
#         tf.keras.Sequential([layers.GlobalMaxPooling2D()]),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    
    optimizer = optimizers.Adam(learning_rate=0.1)
    loss = 'sparse_categorical_crossentropy'
    metrics = 'accuracy'
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

def init_member(parent=-1, gen=0, model=None):
    global MEMBER_RUNNING_ID
    
    m = Member(MEMBER_RUNNING_ID, parent, gen, model or init_model())
    
    MEMBER_RUNNING_ID += 1
    return m

def let_there_be_light():
    START_POPULATION = 5
    
    return list(map(init_member, range(START_POPULATION)))

POPULATION = let_there_be_light()
POPULATION

[[id: 0, parent: 0, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f30247060d0>],
 [id: 1, parent: 1, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f30246ba520>],
 [id: 2, parent: 2, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f30246824c0>],
 [id: 3, parent: 3, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f302468cf10>],
 [id: 4, parent: 4, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f302461ca00>]]

In [5]:
N_ITERATIONS = 3
SURVIVAL_RATE = .7
REPRODUCTION_RATE = .5

def train_member(m):
#     result = m.model.fit(train_ds.batch(2).take(1))
    result = m
    return result

def natural_selection(p):
    def partition(items, size):
        shuffle(items) # In-place shuffle
        left, right = items[:size], items[size:]
        return left, right
    
    mc = len(p) # Member Count
    sc = floor(mc*SURVIVAL_RATE) # Survivals
    survivals, deaths = partition(list(range(mc)), sc) # Survivals and deaths
    
    # Transform survival and death indices to members 
    ids_to_members = lambda ids: list(map(lambda idx: p[idx], ids))
    survivals, _ = ids_to_members(survivals), ids_to_members(deaths)

    return survivals

def reproduction(p):
    def birth(parent):
        # Child Attributes
        parent_id, gen = parent.id, parent.gen+1 
        model = mds.clone_model(parent.model)
    
        child = init_member(parent=parent_id, gen=gen, model=model)
        return child
        
    mc = len(p) # Member Count
    pc = floor(mc*REPRODUCTION_RATE) # Parents
    parents = sample(list(range(mc)), pc)
    
    # Transform parent ids to parents
    parents = map(lambda idx: p[idx], parents)
    
    # Update population with new births
    p.extend(map(birth, parents))
    
    return p

def next_evolution(p):
    def update_member_gen(m):
        m.gen += 1
        return m
    
    # Natural Selection
    p = natural_selection(p)
    
#     # Increment Generation
#     p = list(map(update_member_gen, p))
    
    # Reproduction
    p = reproduction(p)
    
    # Evolve
    result = list(map(train_member, p))
    return result

def calculateDaysBetweenDates()

def run(p):
    result = None
    for step, iteration in enumerate(range(N_ITERATIONS)):
        p = next_evolution(p)
        print(step, p)
        result = p

    return result

run(POPULATION)

0 [[id: 2, parent: 2, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f30246824c0>], [id: 4, parent: 4, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f302461ca00>], [id: 1, parent: 1, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f30246ba520>], [id: 5, parent: 1, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302462d3a0>]]
1 [[id: 4, parent: 4, gen: 0, model: <keras.engine.sequential.Sequential object at 0x7f302461ca00>], [id: 5, parent: 1, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302462d3a0>], [id: 6, parent: 4, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302b93d550>]]
2 [[id: 5, parent: 1, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302462d3a0>], [id: 6, parent: 4, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302b93d550>], [id: 7, parent: 5, gen: 2, model: <keras.engine.sequential.Sequential object at 0x7f302464f970>]]


[[id: 5, parent: 1, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302462d3a0>],
 [id: 6, parent: 4, gen: 1, model: <keras.engine.sequential.Sequential object at 0x7f302b93d550>],
 [id: 7, parent: 5, gen: 2, model: <keras.engine.sequential.Sequential object at 0x7f302464f970>]]